# Montage Generator

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from codex import config as codex_config
from codex import io as codex_io
from codex.ops import tile_crop, tile_generator
import os.path as osp

exp_dir = 'F:\\20180424-d16-tcells-DAPI_CD3_CD8'
config_dir = osp.join(exp_dir, 'config')
output_dir = osp.join(exp_dir, 'output')
res_image = osp.join(exp_dir, 'results', 'montage.tif')

# exp_dir = 'F:\\2018-01-09-Run0-Clean'
# config_dir = exp_dir
# output_dir = 'F:\\2018-01-09-Run0-Clean-out-none\\1-Processor'
# res_image = 'F:\\ComparisonResults\\hammerlab_lung_sample\\montage.tif'

In [2]:
config = codex_config.load(config_dir)
config.channel_names

['DAPI', 'CD3', 'CD8']

In [3]:
config.n_tiles_per_region, config.tile_width, config.tile_height

(225, 1344, 1008)

In [4]:
def load_montage(icyc=0, iz=0, ich=0, region=0, bw=1):
    tiles = []
    tile_indexes = list(range(config.n_tiles_per_region))
    tw, th = config.tile_width, config.tile_height
    rw, rh = config.region_width, config.region_height
    img = np.zeros((rh * th, rw * tw))
    for tile_index in tile_indexes:
        tx, ty = config.get_tile_coordinates(tile_index)
        path = codex_io.get_processor_img_path(region, tx, ty)
        
        tile = codex_io.read_tile(osp.join(output_dir, path), config)
        tile = tile[icyc, iz, ich, :, :]
        
        # Highlight borders
        if bw > 0:
            tile[0:bw, :] = 0
            tile[-bw:, :] = 0
            tile[:, 0:bw] = 0
            tile[:, -bw:] = 0
        
        # Add to montage
        img[(ty * th):((ty + 1) * th), (tx * tw):((tx + 1) * tw)] = tile
    return img

In [5]:
channel = 0
img = load_montage(ich=channel, bw=20)
img.shape

(15120, 20160)

In [6]:
codex_io.save_image(res_image, img.astype(np.float32))
res_image

'F:\\20180424-d16-tcells-DAPI_CD3_CD8\\results\\montage.tif'

In [8]:
# fig, ax = plt.subplots()
# fig.set_size_inches(12, 12)
# ax.imshow(img)